#### Import and DataReading

In [176]:
import pandas as pd
from sklearn import preprocessing
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
dataframe=pd.read_csv("pr_train.csv")


### Data pre-procesing

In [177]:
def changes(df):
    df.drop(columns=['hubpincode','memcreateddate','srno','mobilepurchasedate',"claimtype"],inplace=True)
    df.claimintdate = df.claimintdate.map(lambda x:x+" 00:00:00")
    df.claimintdate = pd.to_datetime(df.claimintdate).astype('int64')
    df.pickupStartdate = df.pickupStartdate.map(lambda x:x+" 00:00:00")
    df.pickupStartdate = pd.to_datetime(df.pickupStartdate).astype('int64')
    labelEncoder = LabelEncoder()
    df['mobmake'] = labelEncoder.fit_transform(df['mobmake'])
    df.custpincodecategory = labelEncoder.fit_transform(df.custpincodecategory)
    return df
#dataframe['hubpincode']= label_encoder.fit_transform(dataframe['hubpincode']) 
# #dataframe['mobmake']= label_encoder.fit_transform(dataframe['mobmake'])
# dataframe["date"]=dataframe.pickupStartdate-dataframe.claimintdate


In [178]:
dataframe1=changes(dataframe)
dataframe.head()

,mobmake,hubid,pickuppartnerid,deliverypartnerid,insuranceCompanyId,custpincodecategory,custpincode,claimintdate,servicecntrid,pickupStartdate,tat_in_days
0,12,7,233,233,131,1,500001,1518048000000000000,4403,1518480000000000000,5
1,8,10,233,233,131,1,226022,1517702400000000000,4444,1518048000000000000,12
2,8,2,233,233,228,1,110018,1517616000000000000,4388,1518393600000000000,9
3,11,2,233,233,228,2,124001,1517702400000000000,4092,1518566400000000000,9
4,12,21,233,233,131,1,201301,1518134400000000000,4580,1518566400000000000,8


In [191]:
# names=dataframe.columns
# scaler=StandardScaler() 
# dataframe = scaler.fit_transform(dataframe)
# dataframe=pd.DataFrame(dataframe,columns=names)

### GradientBoostingRegressor

In [193]:
feature_cols=['mobmake','hubid','pickuppartnerid','deliverypartnerid','insuranceCompanyId',
              'custpincodecategory','custpincode','claimintdate','servicecntrid','pickupStartdate' ]
X=dataframe[feature_cols]
y=dataframe['tat_in_days']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=1)
from sklearn.ensemble import GradientBoostingRegressor
regressor1=GradientBoostingRegressor(n_estimators=500)
regressor1.fit(X_train, y_train)  
#list(zip(feature_cols,linerg.coef_))
y_pred=regressor1.predict(X_test)
import numpy as np
print(np.sqrt(metrics.mean_squared_error(y_test,y_pred)))

[-0.31197928 -0.89389426  0.53720331 ...  0.31606765 -0.1279806
 -0.16306357]
0.8504719528647733


### Random forest Regressor

In [167]:
feature_cols=['mobmake','hubid','pickuppartnerid','deliverypartnerid','insuranceCompanyId',
              'custpincodecategory','custpincode','claimintdate','servicecntrid','pickupStartdate' ]
X=dataframe[feature_cols]

y=dataframe['tat_in_days']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=1)
from sklearn.ensemble import RandomForestRegressor 
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0) 
regressor.fit(X_train, y_train)  
#list(zip(feature_cols,linerg.coef_))
y_pred=regressor.predict(X_test)

import numpy as np
print(np.sqrt(metrics.mean_squared_error(y_test,y_pred)))

4.51847359366983


### Predicting test dataset with GradientBoostingRegressor

In [188]:
data_test=pd.read_csv("pr_test.csv")
data_test1=changes(data_test)
data_test1=scaling(data_test1)
X_test=data_test1[feature_cols]
y_pred=regressor1.predict(X_test)
data_test2=pd.read_csv("pr_test.csv")
data_test2["predictions"]=y_pred.astype(int)
data_test2.to_csv("data_test")